#### This notebook provide an example of training a Random Forest model in H2O.

In [ ]:
import numpy as np
import pandas as pd
import h2o
from h2o.estimators import H2ORandomForestEstimator


In [ ]:

train = pd.read_csv('../input/ncaaw-2021-features-only/tourney.csv')
test = pd.read_csv('../input/ncaaw-2021-features-only/test.csv')

In [ ]:
train.shape

In [ ]:
h2o.init()

In [ ]:

h2o_train_data = h2o.H2OFrame(train)
h2o_test_data = h2o.H2OFrame(test)

In [ ]:
num_folds = 10
training_columns = list(train.drop(['Season','TeamID1','TeamID2','result'], axis=1).columns)

# Define model
model = H2ORandomForestEstimator(ntrees=1000,
                                 max_depth=50,
                                 nfolds=num_folds,
                                 seed=1234)


# Train model
model.train(x=training_columns,
            y='result',
            training_frame=h2o_train_data)

In [ ]:
# predictions on test sample
test_preds = model.predict(test_data=h2o_test_data).as_data_frame()['predict'].values

In [ ]:
# Saving predictions
test = pd.read_csv('../input/ncaaw-2021-features-only/test.csv')
WSampleSubmission = pd.read_csv('../input/ncaaw-march-mania-2021/WSampleSubmissionStage1.csv')

idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:] # (return game!)

pred = pd.concat([test.ID, pd.Series(test_preds)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = WSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub.to_csv('submission.csv', index=False)

**Any comment?**

